In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_zhishi_caijing=pd.DataFrame(columns=columns)
channel_zhishi_caijing

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[7]/div/a/span').click()  #选择“知识”频道

In [5]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[7]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'知识'

In [6]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[4]/a').click()  #选择“科学科普”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[4]/a').text
sub_channel

'财经'

In [7]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [12]:
print(datetime.datetime.now())

for month in range(7,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_zhishi_caijing.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-22 07:44:39.044804


<ipython-input-12-16d939e3319f>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-12-16d939e3319f>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-12-16d939e3319f>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1Xt4y1Q7ZA视频属性的抓取
BV号为1Xt4y1Q7ZA的视频属性抓取结束


<ipython-input-12-16d939e3319f>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1kK411H7kz视频属性的抓取
BV号为1kK411H7kz的视频属性抓取结束
开始BV号为1LK4y1x7mJ视频属性的抓取
BV号为1LK4y1x7mJ的视频属性抓取结束
开始BV号为1DT4y177EJ视频属性的抓取
BV号为1DT4y177EJ的视频属性抓取结束
开始BV号为1b54y1S7xP视频属性的抓取
BV号为1b54y1S7xP的视频属性抓取结束
开始BV号为1UZ4y1M7SR视频属性的抓取
BV号为1UZ4y1M7SR的视频属性抓取结束
开始BV号为1rz411e71c视频属性的抓取
BV号为1rz411e71c的视频属性抓取结束
开始BV号为1Gp4y1S7bZ视频属性的抓取
BV号为1Gp4y1S7bZ的视频属性抓取结束
开始BV号为14p4y1U7Gs视频属性的抓取
BV号为14p4y1U7Gs的视频属性抓取结束
开始BV号为1F5411e7QM视频属性的抓取
BV号为1F5411e7QM的视频属性抓取结束
开始BV号为12V411B7Nf视频属性的抓取
BV号为12V411B7Nf的视频属性抓取结束
开始BV号为1sk4y1q7cQ视频属性的抓取
BV号为1sk4y1q7cQ的视频属性抓取结束
开始BV号为1HK411J7EN视频属性的抓取
BV号为1HK411J7EN的视频属性抓取结束
开始BV号为1Pf4y1d7Dk视频属性的抓取
BV号为1Pf4y1d7Dk的视频属性抓取结束
开始BV号为1VD4y1U7Le视频属性的抓取
BV号为1VD4y1U7Le的视频属性抓取结束
开始BV号为1Si4y1G76E视频属性的抓取
BV号为1Si4y1G76E的视频属性抓取结束
开始BV号为1Wv411q7qR视频属性的抓取
BV号为1Wv411q7qR的视频属性抓取结束
开始BV号为1sC4y1b78k视频属性的抓取
BV号为1sC4y1b78k的视频属性抓取结束
开始BV号为1rv411B7Lk视频属性的抓取
BV号为1rv411B7Lk的视频属性抓取结束
开始BV号为1Tp4y1S72G视频属性的抓取
BV号为1Tp4y1S72G的视频属性抓取结束
开始BV号为1d5411Y7eR视频属性的抓取
BV号为1d5411Y7eR的视频属性抓取结束
开始BV号为1dZ4y1T7c4视频属性的抓取
BV号为1dZ4y1T7c4的视

开始BV号为1i54y127nu视频属性的抓取
BV号为1i54y127nu的视频属性抓取结束
开始BV号为19V411z7cj视频属性的抓取
BV号为19V411z7cj的视频属性抓取结束
开始BV号为1Wk4y1y7TH视频属性的抓取
BV号为1Wk4y1y7TH的视频属性抓取结束
开始BV号为1Y54y127EN视频属性的抓取
BV号为1Y54y127EN的视频属性抓取结束
开始BV号为12D4y127rJ视频属性的抓取
BV号为12D4y127rJ的视频属性抓取结束
开始BV号为17K411T7pw视频属性的抓取
BV号为17K411T7pw的视频属性抓取结束
开始BV号为1bv411i7gU视频属性的抓取
BV号为1bv411i7gU的视频属性抓取结束
开始BV号为1s54y1v7YU视频属性的抓取
BV号为1s54y1v7YU的视频属性抓取结束
开始BV号为13V411U7Sy视频属性的抓取
BV号为13V411U7Sy的视频属性抓取结束
开始BV号为1dy4y1C7Z9视频属性的抓取
BV号为1dy4y1C7Z9的视频属性抓取结束
开始BV号为1fh411R7qx视频属性的抓取
BV号为1fh411R7qx的视频属性抓取结束
开始BV号为1AK411T7ig视频属性的抓取
BV号为1AK411T7ig的视频属性抓取结束
开始BV号为1864y1c7Qe视频属性的抓取
BV号为1864y1c7Qe的视频属性抓取结束
开始BV号为1Gh411R7Wv视频属性的抓取
BV号为1Gh411R7Wv的视频属性抓取结束
开始BV号为1uK411T7E9视频属性的抓取
BV号为1uK411T7E9的视频属性抓取结束
开始BV号为1GD4y1m7tu视频属性的抓取
BV号为1GD4y1m7tu的视频属性抓取结束
开始BV号为1uV411U7Lu视频属性的抓取
BV号为1uV411U7Lu的视频属性抓取结束
开始BV号为1vf4y197Mq视频属性的抓取
BV号为1vf4y197Mq的视频属性抓取结束
开始BV号为1op4y1v71r视频属性的抓取
BV号为1op4y1v71r的视频属性抓取结束
开始BV号为1iA411n7sc视频属性的抓取
BV号为1iA411n7sc的视频属性抓取结束
开始BV号为1Ev411i7j3视频属性的抓取
BV号为1Ev411i7j3的视

<ipython-input-12-16d939e3319f>:190: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1rK411K7Eu视频属性的抓取
BV号为1rK411K7Eu的视频属性抓取结束
开始BV号为1Ma4y1j7Lw视频属性的抓取
BV号为1Ma4y1j7Lw的视频属性抓取结束
开始BV号为1Ef4y1D7cA视频属性的抓取
BV号为1Ef4y1D7cA的视频属性抓取结束
开始BV号为1Rp4y1e73J视频属性的抓取
BV号为1Rp4y1e73J的视频属性抓取结束
开始BV号为1TA411J7EY视频属性的抓取
BV号为1TA411J7EY的视频属性抓取结束
开始BV号为11i4y177HX视频属性的抓取
BV号为11i4y177HX的视频属性抓取结束
开始BV号为1K64y1F7Bi视频属性的抓取
BV号为1K64y1F7Bi的视频属性抓取结束
开始BV号为1Gi4y1E7G7视频属性的抓取
BV号为1Gi4y1E7G7的视频属性抓取结束
开始BV号为1Bk4y1y73i视频属性的抓取
BV号为1Bk4y1y73i的视频属性抓取结束
开始BV号为1xK4y1e7Fm视频属性的抓取
BV号为1xK4y1e7Fm的视频属性抓取结束
开始BV号为1MV411m7ek视频属性的抓取
BV号为1MV411m7ek的视频属性抓取结束
开始BV号为13y4y1y7ko视频属性的抓取
BV号为13y4y1y7ko的视频属性抓取结束
开始BV号为1Vh411R7aM视频属性的抓取
BV号为1Vh411R7aM的视频属性抓取结束
开始BV号为17V411m7vi视频属性的抓取
BV号为17V411m7vi的视频属性抓取结束
开始BV号为1654y1y7HU视频属性的抓取
BV号为1654y1y7HU的视频属性抓取结束
开始BV号为1KD4y1o7uA视频属性的抓取
BV号为1KD4y1o7uA的视频属性抓取结束
开始BV号为1Vt4y1q7rD视频属性的抓取
BV号为1Vt4y1q7rD的视频属性抓取结束
开始BV号为1sD4y1o7wC视频属性的抓取
BV号为1sD4y1o7wC的视频属性抓取结束
开始BV号为1b54y1m786视频属性的抓取
BV号为1b54y1m786的视频属性抓取结束
开始BV号为1Cz4y1f74p视频属性的抓取
BV号为1Cz4y1f74p的视频属性抓取结束
开始BV号为1U5411b7Xn视频属性的抓取
BV号为1U5411b7Xn的视

<ipython-input-12-16d939e3319f>:187: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1rT4y1c7SJ视频属性的抓取
BV号为1rT4y1c7SJ的视频属性抓取结束
开始BV号为1Ct4y1v77w视频属性的抓取
BV号为1Ct4y1v77w的视频属性抓取结束
开始BV号为1BK411N7dR视频属性的抓取
BV号为1BK411N7dR的视频属性抓取结束
开始BV号为1Mv411k7Jm视频属性的抓取
BV号为1Mv411k7Jm的视频属性抓取结束
开始BV号为1iz4y1o7WZ视频属性的抓取
BV号为1iz4y1o7WZ的视频属性抓取结束
开始BV号为1Yi4y1774g视频属性的抓取
BV号为1Yi4y1774g的视频属性抓取结束
开始BV号为1u54y1r73Q视频属性的抓取
BV号为1u54y1r73Q的视频属性抓取结束
开始BV号为1wZ4y1V7c4视频属性的抓取
BV号为1wZ4y1V7c4的视频属性抓取结束
开始BV号为1iz4y1o7D3视频属性的抓取
BV号为1iz4y1o7D3的视频属性抓取结束
开始BV号为1Ca4y1L7x6视频属性的抓取
BV号为1Ca4y1L7x6的视频属性抓取结束
开始BV号为15T4y1F7NP视频属性的抓取
BV号为15T4y1F7NP的视频属性抓取结束
开始BV号为1g54y1r7cz视频属性的抓取
BV号为1g54y1r7cz的视频属性抓取结束
开始BV号为1Yp4y1Y7Xa视频属性的抓取
BV号为1Yp4y1Y7Xa的视频属性抓取结束
开始BV号为1eD4y1R7iA视频属性的抓取
BV号为1eD4y1R7iA的视频属性抓取结束
开始BV号为1qA411J7yg视频属性的抓取
BV号为1qA411J7yg的视频属性抓取结束
开始BV号为1PA411E782视频属性的抓取
BV号为1PA411E782的视频属性抓取结束
开始BV号为1y54y1k7jz视频属性的抓取
BV号为1y54y1k7jz的视频属性抓取结束
开始BV号为1TK411A7NU视频属性的抓取
BV号为1TK411A7NU的视频属性抓取结束
开始BV号为1qz4y1C7GC视频属性的抓取
BV号为1qz4y1C7GC的视频属性抓取结束
开始BV号为1Tv411k7gF视频属性的抓取
BV号为1Tv411k7gF的视频属性抓取结束
开始BV号为1dr4y1c7Gp视频属性的抓取
BV号为1dr4y1c7Gp的视

In [11]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-11-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [13]:
channel_zhishi_caijing.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
1月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100


In [14]:
channel_zhishi_caijing.to_json('channel_zhishi_caijing.json')